### 손글씨 이미지 데이터 MNIST

- 인코딩된 바이너리 데이터를 디코딩하여 처리하는 방식 확인
- 지도 학습
- 학습용 데이터는 6만개, 테스트 데이터는 1만개
- 결론
    - 학습 후 새로운 데이터 입력시 판별
    - 0~9까지의 손글씨 이미지를 판별
    - 데이터는 url을 직접 획득해서, 원하는 곳에 다운로드 시키겠다

### 절차

|No|단계|내용|
|:---:|:---|:---|
|1|연구목표|- 손글씨 이미지(0-9)를 학습시켜서, 새로운 손글씨 이미지를 판별해 내는 머신러닝 모델을 구축<br>- 압축된 이미지를 압축해제<br>- 인코딩된 데이터를 디코딩 처리<br>- 28X28로 구성된 픽셀 이미지 데이터를 벡터화 처리<br>- 시스템 통합의 결과를 복 연구 목표를 설정해야 하지만 시스템 통합을 생략하므로 이부분은 생략|
|2|데이터획득/수집|- http://yann.lecun.com/exdb/mnist/ 접속<br>- web scraping을 통해서 데이터의 URL 획득<br>- 지정된 위치에 다운로드->압축해제|
|3|데이터준비/전처리|- 디코딩(내부구조를 알수 있는 인코딩 문서(MNIST Database) 필요)<br>- 에디언(Endian)처리<br>- 벡터화 처리|
|4|데이터탐색/통찰/시각화분석|- 생략|
|5|데이터모델링(머신러닝모델링)|- 분류 알고리즘 사용<br>- 알고리즘 선택->훈련용 데이터와 학습용 데이터 나눔->학습->예측->평가|
|6|시스템통합|- 생략|

### 2. 데이터 획득/수집

- 모듈 준비

In [1]:
import urllib.request as req
from bs4 import BeautifulSoup

- web scraping

In [2]:
rootUrl = 'http://yann.lecun.com/exdb/mnist/'
soup = BeautifulSoup(req.urlopen(rootUrl),'html5lib')

- 4개의 url 획득

In [3]:
#모든 요소 tt중에 상위 4개가 링크
for tt in soup.findAll('tt')[:4]:
    print(tt.a.string)

train-images-idx3-ubyte.gz
train-labels-idx1-ubyte.gz
t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz


In [4]:
files=[tt.a.string for tt in soup.findAll('tt')[:4]]
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

- 다운로드>압축해제 <- 반복작업

In [2]:
import os, os.path, gzip

In [6]:
savePath = './data/mnist'
if not os.path.exists(savePath):
    os.makedirs(savePath)

In [11]:
#tqdm:진행율을 보여주는 모듈
from tqdm import tqdm_notebook
for file in tqdm_notebook(files):
    print('소스',rootUrl + file)
    local_path='%s/%s' % (savePath,file)
    print('대상',local_path)
    
    #웹상에 존재하는 리소스를 로컬 디스크상에 직접 저장
    req.urlretrieve(rootUrl + file,local_path)

소스 http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
대상 ./data/mnist/train-images-idx3-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
대상 ./data/mnist/train-labels-idx1-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
대상 ./data/mnist/t10k-images-idx3-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
대상 ./data/mnist/t10k-labels-idx1-ubyte.gz



In [13]:
#압축해제
for file in tqdm_notebook(files):
    #원본파일의 경로
    ori_gzip_file='%s/%s' % (savePath,file)
    print(ori_gzip_file)
    #압축해제 파일의 경로
    target_file='%s/%s' % (savePath,file[:-3])
    print(target_file)
    #압축해제
    #gzip의 파일오픈 -> 읽기 -> 쓰기
    with gzip.open(ori_gzip_file,'rb') as fg:
        #읽기(압축해제를 수행했다)
        tmp = fg.read()
        #쓰기:일반파일로 기록
        with open(target_file,'wb') as f:
            f.write(tmp)

./data/mnist/train-images-idx3-ubyte.gz
./data/mnist/train-images-idx3-ubyte
./data/mnist/train-labels-idx1-ubyte.gz
./data/mnist/train-labels-idx1-ubyte
./data/mnist/t10k-images-idx3-ubyte.gz
./data/mnist/t10k-images-idx3-ubyte
./data/mnist/t10k-labels-idx1-ubyte.gz
./data/mnist/t10k-labels-idx1-ubyte



### 3. 데이터준비/전처리

- 디코딩(내부구조를 알수 있는 인코딩 문서(MNIST Database) 필요)
- 에디언(Endian)처리(TCP/IP상에서 통신 수행시 중요)
    - 컴퓨터 메모리와 같은 1차원 공간에 여러개의 연속된 데이터를 배열하는 방법
    - 종류:바이트를 배치하는 오더(순서)를 앞에서부터 혹은 뒤에서부터 채우는가
        - 0x12345678
        - 빅 에디언 : 값을 앞에서부터 채운다
            0x12 0x34 0x56 0x78
        - 리틀 에디언 : 값을 뒤에서부터 채운다
            0x78 0x56 0x34 0x12
        - 위의 예는 정수값(4byte)을 예를 든것이고, 단지 값이 어떻게 기록됐는지만 이해하고, 그대로 값을 복원할 수 있으면 끝
- 벡터화 처리

- label file
    - magic number : 4byte -> 에디안 체크
    - label 수 : 4byte -> 에디안
    - label 데이터 : 1byte -> 0~9 값
    - 크기 = 4 + 4 + label수*1byte = 8 + 60000 = 60008byte
- image file
    - magic number : 4byte -> 에디안 체크
    - 손글시 이미지 개수 : 4byte -> 에디안 체크
    - 가로크기(픽셀수) : 4byte -> 에디안 체크
    - 세로크기(픽셀수 : 4byte -> 에디안 체크
    - 픽셀값 한개 한개 : unsigned 1byte(=8bit)(0~2^8-1 : 0~255(0xFF))

In [3]:
#원데이터의 구조를 이해했으니, 구조에 맞춰서 데이터를 디코딩한다
import struct 

In [19]:
#테스트용 레이블 파일 처리
label_f=open('./data/mnist/t10k-labels-idx1-ubyte','rb')
#바이너리 데이터는 헤더부터 읽어서 데이터의 유효성이나 종류를 인지
#MNIST 파일은 규격서에 high(빅) edian으로 수치값을 기술했다라고 확인되었다.
#label파일은 헤더가 4+4=8byte이다(규격서기준)
#high(빅) edian ->>
#4 -> I(i의 대문자)
#헤더 정보 추출
magic_number, label_count = struct.unpack('>II',label_f.read(8))
#magic_number : 2019 -> 레이블 파일이다
#label_count : 1000 -> 데이터의 개수(레이블의 개수, 답의 개수)
print(magic_number, label_count)

image_f=open('./data/mnist/t10k-images-idx3-ubyte','rb')
magic_number2, image_count, row, col = struct.unpack('>IIII',image_f.read(16))
print(magic_number2, image_count, row, col)

#헤더 정보를 기초로 반복 작업수행 : 정답추출, 이미지 추출
#헤더크기=16+이미지1개데이터크기(28*28)*이미지총개수(10000)
print('이미지파일의크기',4+4+4+4+10000*28*28,'bytes')

#헤더 정보를 기초로 반복작업:정답추출, 이미지 추출
for idx in range(image_count):
    #정답 추출 : label_f를 통해서 1byte읽는다. 단, unsigned(부호없는, 양수) byte -> 'B'
    label_tmp = struct.unpack('B', label_f.read(1)) #파일을 읽으면 읽은 만큼 누적으로 커서(파일포인터)위치이동
    label=label_tmp[0]
    print(label)
    #이미지 추출
    
    #이미지 데이터의 벡터화

image_f.close()
label_f.close()

2049 10000
2051 10000 28 28
이미지파일의크기 7840016 bytes
7


In [21]:
label_f=open('./data/mnist/t10k-labels-idx1-ubyte','rb')
image_f=open('./data/mnist/t10k-images-idx3-ubyte','rb')

magic_number, label_count = struct.unpack('>II',label_f.read(8))
magic_number2, image_count, row, col = struct.unpack('>IIII',image_f.read(16))

#이미지 1개당 크기
pixels=row*col #28x28
for idx in range(image_count):
    label_tmp = struct.unpack('B', label_f.read(1))
    label=label_tmp[0]
    #이미지 추출 -> 바이너리 데이터를 읽는다 -> 에디안은 관계 없음
    binarryData = image_f.read(pixels)
#     print(type(binarryData), len(binarryData),binarryData)
    #픽셀값 하나하나를 문자열로 만들어서 리스트에 담는다
    #바이너리 값은 0~255의 문자열로 변경했다
    strData = list(map(lambda x:str(x), binarryData))
    print(strData)
    # csv에 한줄의 데이터로 기록 -> 1 + 784개를 기록 -> 1개의 데이터 표현
    # 구분자 ,
    
    #pgm파일로 dump처리해서 확인(데이터의 유효성 확인)
    
    
    #이미지 데이터의 벡터화
    break
    pass

image_f.close()
label_f.close()

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

In [23]:
label_f=open('./data/mnist/t10k-labels-idx1-ubyte','rb')
image_f=open('./data/mnist/t10k-images-idx3-ubyte','rb')
#csv 파일 오픈
csv_f=open('./data/mnist/t10k.csv','w',encoding='utf-8')
magic_number, label_count = struct.unpack('>II',label_f.read(8))
magic_number2, image_count, row, col = struct.unpack('>IIII',image_f.read(16))

pixels=row*col 
for idx in range(image_count):
    label_tmp = struct.unpack('B', label_f.read(1))
    label=label_tmp[0]
    binarryData = image_f.read(pixels)
    strData = list(map(lambda x:str(x), binarryData))
    # csv에 한줄의 데이터로 기록 -> 1 + 784개를 기록 -> 1개의 데이터 표현
    csv_f.write(str(label)+',')
    csv_f.write(','.join(strData) + '\n')
    #pgm파일로 dump처리해서 확인(데이터의 유효성 확인)
    if idx== 0 : #한번만 수행
        with open('./data/mnist/%s.pgm' % label,'w', encoding='utf-8') as f:
            f.write('P2 28 28 255\n' + ' '.join(strData))
    #이미지 데이터의 벡터화
    break
    pass

image_f.close()
label_f.close()
csv_f.close()

In [4]:
def decoding_mnist_rawData(dataStyle='train',maxCount=0):
    label_f=open(f'./data/mnist/{dataStyle}-labels-idx1-ubyte','rb')
    image_f=open(f'./data/mnist/{dataStyle}-images-idx3-ubyte','rb')
    csv_f=open(f'./data/mnist/{dataStyle}.csv','w',encoding='utf-8')
    magic_number, label_count = struct.unpack('>II',label_f.read(8))
    magic_number2, image_count, row, col = struct.unpack('>IIII',image_f.read(16))

    if maxCount>=image_count:
        print('개수의 범위를 넘었습니다. 최소 %s개 이내' % image_count)
        return
    elif maxCount== -1:
        maxCount = image_count
    elif maxCount <-1:
        print('개수의 범위를 넘었습니다. 최소 %s개 이내' % image_count)
        return
    
    pixels=row*col 
    for idx in (range(maxCount)):
#         if idx >= maxCount : break
        label_tmp = struct.unpack('B', label_f.read(1))
        label=label_tmp[0]
        binarryData = image_f.read(pixels)
        strData = list(map(lambda x:str(x), binarryData))
        csv_f.write(str(label)+',')
        csv_f.write(','.join(strData) + '\n')

    image_f.close()
    label_f.close()
    csv_f.close()
    


In [18]:
decoding_mnist_rawData(dataStyle='train',maxCount=58000)
decoding_mnist_rawData(dataStyle='t10k',maxCount=9000)

#### [M1] 데이터 품질 향상

- 정확도를 96%목표로 머신러닝 모델을 개선

    [사전조치]
    - 머신러닝 모델을 이용하여 예측시 정확도가 떨어지면 데이터의 품질, 양을 검토한다
    - 양을 점차적으로 늘린다
        - 데이터의 개수를 늘리거나, 비율을 조정(훈련:테스트=75:25)
    - 품질을 향상시킨다
        - 정규화
        - 차후에 적용가능한 내용 : PCA같은 비지도 학습의 차원축소(피처의 수를 줄인다)
        
    [모델개선조치]
    - 알고리즘 교체
    - 하이퍼파라미터 튜닝
    - 파이프라인을 이용한 전처리기를 활용(품질향상)하여 향상
    - 이런 교차 검증법을 활용하여 성능향상을 도모한다
    - 이런 것들의 검증은 ROC곡선, AUC값 등으로 확인할 수도 있고, 교차 검증법의 결과로 확인 가능

### 4	데이터탐색/통찰/시각화분석

- skip

In [6]:
# 다음 함수를 만든다 load_csv(dataType='train')
# 현재 csv파일은 t10k.csv, train.csv
# 출력 데이터 : csv 파일명
# 입력 데이터 : {'labels':[],'images':[[]]}

def load_csv(dataType='train'):
    labels=list()
    images=list()
    with open(f'./data/mnist/{dataType}.csv','r') as f:
        for line in f:
            tmp=line.split(',')
            labels.append(int(tmp[0]))
            images.append(list(map(lambda x:int(x),tmp[1:])))
    return {'labels':labels,'images':images}


In [38]:
def load_csv(dataType='train'):
    f = open(f'./data/mnist/{dataType}.csv', 'r')
    
    labels = list()
    images = list()
    
    while True:
        row = f.readline()
        if not row: break
        labels.append(int(row.split(',')[0]))
        images.append(list(map(lambda x:int(x),row.split(',')[1:])))
    f.close()
    return { 'labels':labels, 'images':images }

In [19]:
train=load_csv()
test=load_csv('t10k')

### 5	데이터모델링(머신러닝모델링)

- 지도학습 데이터이므로, 정확도를 통해서 평가를 1차로 수행

In [20]:
from sklearn import svm, model_selection, metrics

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std_train=scaler.fit_transform(train['images'])

In [22]:
#2. 알고리즘 생성
clf=svm.SVC()

In [17]:
#3. 데이터분류(위에서 완료)

In [23]:
len(train['images']),len(train['labels'])

(58000, 58000)

In [ ]:
#4. 학습
clf.fit(train['images'], train['labels'])

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [16]:
#5. 예측
predict=clf.predict(scaler.transform(test['images']))

In [17]:
#6. 평가
metrics.accuracy_score(test['labels'],predict)

0.136

In [15]:
#7. 오차행렬(혼돈행렬)을 이용한 평가
clf_report=metrics.classification_report(test['labels'],predict)
print(clf_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       219
           1       0.11      1.00      0.21       287
           2       0.00      0.00      0.00       276
           3       0.00      0.00      0.00       254
           4       0.00      0.00      0.00       275
           5       0.00      0.00      0.00       221
           6       0.00      0.00      0.00       225
           7       0.00      0.00      0.00       257
           8       0.00      0.00      0.00       242
           9       0.00      0.00      0.00       244

    accuracy                           0.11      2500
   macro avg       0.01      0.10      0.02      2500
weighted avg       0.01      0.11      0.02      2500



C:\Users\admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
